In [67]:
# -*- coding: utf-8 -*-
# Dosya Yolu Test Aracı: verimgoren_cli.py dosyasındaki harici veri yollarını kontrol ederek doğru
# KÖK DİZİNİ (Base Directory) bulmanıza yardımcı olur.

import os
import sys
from pathlib import Path
from typing import List, Tuple

# =================================================================
# 0) VERİ KLASÖR YAPISI ŞABLONU
# =================================================================
FILE_REL_PATHS: List[Tuple[str, str]] = [

    ("CROPS_CSV (Bitki Parametreleri)", "notebooks/VerimGoren_Bitki_Parametreleri_Tam.csv"),

    ("CLIMATE_CSV (İklim Verisi)", "notebooks/data/climate/merged_climate_data.csv"),

    ("ELEV_TIF (Rakım Raster)", "../VerimGören/data/processed/srtm_turkiye_cropped.tif"),

    ("LIGHT_TIF (Gece Işığı Raster)", "../VerimGören/data/processed/viirs_light_2024_turkey.tif"),

    ("HWSD_MDB (Toprak Veritabanı)", "notebooks/hwsd_data/HWSD.mdb"),

    ("HWSD_RAS (Toprak Raster)", "notebooks/hwsd_data/hwsd.bil"),
]

# =================================================================
# 1) ÇÖZÜM YÖNTEMLERİ TESTİ - LÜTFEN SADECE BİRİNİ SEÇİN (1, 2, 3 veya 4)
# =================================================================
# Yeni bir seviye yukarı çıkma modu eklendi: MODE_SELECT = 4
MODE_SELECT = 4 

# --- 1. YÖNTEM: MANUEL MUTLAK YOL GİRİŞİ ---
MANUAL_BASE_DIR = "C:\\KENDİ_PROJE_KÖKÜNÜZÜ_BURAYA_GİRİN" 

# Değişkenlerin tanımlanması
base_dir = Path.cwd()
TEST_MODE = ""

if MODE_SELECT == 1:
    if MANUAL_BASE_DIR == "C:\\KENDİ_PROJE_KÖKÜNÜZÜ_BURAYA_GİRİN":
        print("❌ HATA: Manuel yol seçildi (MODE_SELECT = 1) ancak MANUAL_BASE_DIR ayarlanmadı.")
        sys.exit(1)
    base_dir = Path(MANUAL_BASE_DIR)
    TEST_MODE = "MANUEL MUTLAK YOL"
elif MODE_SELECT == 2:
    base_dir = Path.cwd() 
    TEST_MODE = "GÖRECELİ (Çalışma Dizini - Path.cwd())"
elif MODE_SELECT == 3:
    # Betiğin kendi dizinine göre arar
    try:
        base_dir = Path(__file__).resolve().parent
    except NameError:
        print("⚠️ Uyarı: '__file__' değişkeni bulunamadı. Çalışma dizini kullanılıyor.")
        base_dir = Path.cwd()
    TEST_MODE = "GÖRECELİ (Betiğin Klasörüne Göre - Path(__file__).parent)"
elif MODE_SELECT == 4:
    # Çalışma dizininden bir seviye yukarı çıkar (notebooks içinden çalıştırılıyorsa ideal)
    base_dir = Path.cwd().parent
    TEST_MODE = "GÖRECELİ (Çalışma Dizini.parent - Bir Seviye Yukarı)"
else:
    print(f"❌ HATA: Geçersiz MODE_SELECT değeri: {MODE_SELECT}. Lütfen 1, 2, 3 veya 4 seçin.")
    sys.exit(1)


def generate_paths_for_test(base_dir: Path) -> List[Tuple[str, Path]]:
    paths = []
    for name, rel_path_str in FILE_REL_PATHS:
        # Mutlak yolu oluşturmak için base_dir ile göreceli yolu birleştir
        full_path = base_dir / Path(rel_path_str)
        paths.append((name, full_path))
    return paths

def test_file_paths(paths_with_names: List[Tuple[str, Path]], mode: str, base: Path):
    """Tanımlı tüm dosya yollarının varlığını kontrol eder."""
    print("=========================================================")
    print(f"🔍 HARİCİ DOSYA YOLU KONTROLÜ - MOD: {mode}")
    print(f"📦 KÖK DİZİN (Base Dir): {base}")
    print("=========================================================")
    
    all_ok = True
    
    for name, path in paths_with_names:
        if path.exists():
            print(f"✅ {name:<35}: BULUNDU")
        else:
            print(f"❌ {name:<35}: BULUNAMADI")
            print(f"    Denendiği Yol: {path}")
            all_ok = False
            
    print("---------------------------------------------------------")
    if all_ok:
        print("🎉 TÜM DOSYALAR BULUNDU.")
        print("👉 ÇÖZÜM: Artık verimgoren_cli.py dosyasındaki yolları bu Kök Dizinine göre güncelleyebiliriz.")
    else:
        print("🛑 EKSİK VEYA HATALI YOLLAR TESPİT EDİLDİ.")
        print("   Lütfen MODE_SELECT değerini (1, 2, 3 veya 4) değiştirerek farklı yöntemleri deneyin.")
    print("=========================================================")
    return all_ok

if __name__ == "__main__":
    
    try:
        paths_to_test = generate_paths_for_test(base_dir)
        test_file_paths(paths_to_test, TEST_MODE, base_dir)
    except Exception as e:
        print(f"Beklenmedik bir hata oluştu: {type(e).__name__}: {e}")

🔍 HARİCİ DOSYA YOLU KONTROLÜ - MOD: GÖRECELİ (Çalışma Dizini.parent - Bir Seviye Yukarı)
📦 KÖK DİZİN (Base Dir): C:\Users\ataka\Desktop\MEHMET\VerimGören
✅ CROPS_CSV (Bitki Parametreleri)    : BULUNDU
✅ CLIMATE_CSV (İklim Verisi)         : BULUNDU
✅ ELEV_TIF (Rakım Raster)            : BULUNDU
✅ LIGHT_TIF (Gece Işığı Raster)      : BULUNDU
✅ HWSD_MDB (Toprak Veritabanı)       : BULUNDU
✅ HWSD_RAS (Toprak Raster)           : BULUNDU
---------------------------------------------------------
🎉 TÜM DOSYALAR BULUNDU.
👉 ÇÖZÜM: Artık verimgoren_cli.py dosyasındaki yolları bu Kök Dizinine göre güncelleyebiliriz.


In [87]:
# -*- coding: utf-8 -*-
# VerimGören — Konumdan: Tüm Veriler (UI) + Ürün Uygunluk Skoru (0–100)
# Kullanım:
#   python verimgoren_cli.py
#   Konum girin (Google Maps linki YA DA 'lat,lon'):  41.239705,41.915686

import os, re, sys
from pathlib import Path
from typing import Tuple, Optional, Dict
import pandas as pd
import numpy as np

# =========================
# 0) YOLLAR — KENDİ SİSTEMİNE GÖRE GÜNCELLE
# =========================
CROPS_CSV   = R("VerimGoren_Bitki_Parametreleri_Tam.csv")
CLIMATE_CSV = R("data/climate/merged_climate_data.csv")  # 0.5°
ELEV_TIF    = R("../data/processed/srtm_turkiye_cropped.tif")         # opsiyonel
LIGHT_TIF   = R("../data/processed/viirs_light_2024_turkey.tif")      # opsiyonel
HWSD_MDB    = R("hwsd_data/HWSD.mdb")                                # opsiyonel
HWSD_RAS    = R("hwsd_data/hwsd.bil")                                # opsiyonel


# =========================
# 1) Koordinat ayrıştırma (senin yapın)
# =========================
DMS_PATTERN = re.compile(r"(?P<deg>\d{1,3})°(?P<min>\d{1,2})'(?P<sec>[\d\.]+)\"(?P<hemi>[NSEW])")

def dms_to_decimal(deg: float, minute: float, sec: float, hemi: str) -> float:
    sign = -1 if hemi.upper() in ["S", "W"] else 1
    return sign * (abs(deg) + minute / 60.0 + sec / 3600.0)

def parse_latlon_text(text: str) -> Tuple[float, float]:
    text = text.strip()
    parts = re.split(r"[,\s;]+", text)
    parts = [p for p in parts if p]
    if len(parts) < 2:
        raise ValueError("Lütfen 'lat,lon' biçiminde iki sayı girin. Örn: 38.946838, 28.080573")
    lat = float(parts[0]); lon = float(parts[1])
    if not (-90.0 <= lat <= 90.0 and -180.0 <= lon <= 180.0):
        raise ValueError("Geçersiz aralık: enlem [-90,90], boylam [-180,180].")
    return lat, lon

def parse_google_maps_link(text: str) -> Optional[Tuple[float, float]]:
    text = text.strip()
    if "@" in text:
        try:
            after = text.split("@", 1)[1]
            nums = re.split(r"[^-\d\.]+", after)
            nums = [n for n in nums if n]
            lat = float(nums[0]); lon = float(nums[1])
            return lat, lon
        except Exception:
            pass
    dms_hits = DMS_PATTERN.findall(text)
    if len(dms_hits) >= 2:
        (ld, lm, ls, lh), (od, om, os, oh) = dms_hits[0], dms_hits[1]
        lat = dms_to_decimal(float(ld), float(lm), float(ls), lh)
        lon = dms_to_decimal(float(od), float(om), float(os), oh)
        return lat, lon
    return None

def parse_any_location(text: str) -> Tuple[float, float]:
    link = parse_google_maps_link(text)
    if link is not None:
        return link
    return parse_latlon_text(text)

def google_maps_url_from_latlon(lat: float, lon: float, zoom: int = 12) -> str:
    return f"https://www.google.com/maps/@{lat:.6f},{lon:.6f},{zoom}z"

def google_maps_place_url(lat: float, lon: float) -> str:
    return f"https://www.google.com/maps?q={lat:.6f},{lon:.6f}"

# =========================
# 2) Kategoriler & değişken meta (TR başlık + birim)
# =========================
CATEGORY_ORDER = ["Konum", "İklim", "Arazi", "Gece Işığı", "Toprak", "Özet"]

def category_of(key: str) -> str:
    k = key.upper()
    if k in {"USER_LAT","USER_LON","GRID_LAT","GRID_LON","LATITUDE","LONGITUDE"}: return "Konum"
    if k in {"ELEVATION_M"}: return "Arazi"
    if k in {"NIGHT_LIGHT"}: return "Gece Işığı"
    if ("_GRP" in k) or k in {
        "T2M","T2M_MAX","T2M_MIN","T2M_RANGE","T2MDEW","T2MWET","RH2M","QV2M","TQV","PS","SLP",
        "WS2M","WS2M_MAX","WD2M","PRECTOTCORR","TS","TO3","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_PAR_TOT",
        "CLRSKY_SFC_SW_DWN","CLOUD_AMT","CLOUD_AMT_DAY","CLOUD_AMT_NIGHT","CLRSKY_DAYS","DISTANCE_KM"
    }:
        return "İklim"
    if k in {
        "FAO90_DESC","T_USDA_TEX_DESC","S_USDA_TEX_DESC","T_TEXTURE_DESC",
        "T_SAND","T_SILT","T_CLAY","S_SAND","S_SILT","S_CLAY",
        "T_PH_H2O","S_PH_H2O","T_OC","S_OC","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB",
        "T_CACO3","S_CACO3","T_ECE","S_ECE","T_ESP","S_ESP",
        "AWC_MM_PER_M","DRAINAGE_DESC","MU_GLOBAL"
    }:
        return "Toprak"
    if k in {"DISTANCE_KM"}: return "Özet"
    return "Özet"

VAR_META: Dict[str, Dict[str, str]] = {
    # Konum
    "USER_LAT":{"title_tr":"Kullanıcı enlem","unit":"°"},
    "USER_LON":{"title_tr":"Kullanıcı boylam","unit":"°"},
    "GRID_LAT":{"title_tr":"İklim hücresi enlem","unit":"°"},
    "GRID_LON":{"title_tr":"İklim hücresi boylam","unit":"°"},
    "LATITUDE":{"title_tr":"Enlem","unit":"°"},
    "LONGITUDE":{"title_tr":"Boylam","unit":"°"},
    # İklim temel
    "ALLSKY_SFC_PAR_TOT":{"title_tr":"PAR (tümü)","unit":"MJ/m²/gün"},
    "ALLSKY_SFC_SW_DWN":{"title_tr":"Kısa dalga (tümü)","unit":"kWh/m²/gün"},
    "CLRSKY_SFC_SW_DWN":{"title_tr":"Kısa dalga (açık gök)","unit":"kWh/m²/gün"},
    "CLRSKY_DAYS":{"title_tr":"Açık gün sayısı","unit":"gün/ay"},
    "CLOUD_AMT":{"title_tr":"Bulutluluk","unit":"%"},
    "CLOUD_AMT_DAY":{"title_tr":"Bulutluluk (gündüz)","unit":"%"},
    "CLOUD_AMT_NIGHT":{"title_tr":"Bulutluluk (gece)","unit":"%"},
    "QV2M":{"title_tr":"Özgül nem (2 m)","unit":"g/kg"},
    "RH2M":{"title_tr":"Bağıl nem (2 m)","unit":"%"},
    "T2M":{"title_tr":"Sıcaklık (2 m, ort.)","unit":"°C"},
    "T2M_MAX":{"title_tr":"Maks. sıcaklık","unit":"°C"},
    "T2M_MIN":{"title_tr":"Min. sıcaklık","unit":"°C"},
    "T2M_RANGE":{"title_tr":"Günlük sıcaklık aralığı","unit":"°C"},
    "T2MDEW":{"title_tr":"Çiy noktası","unit":"°C"},
    "T2MWET":{"title_tr":"Yaş termometre","unit":"°C"},
    "TQV":{"title_tr":"Kolon su buharı","unit":"kg/m²"},
    "PS":{"title_tr":"Yüzey basıncı","unit":"kPa"},
    "SLP":{"title_tr":"Denize indirgenmiş basınç","unit":"kPa"},
    "WD2M":{"title_tr":"Rüzgar yönü (2 m)","unit":"°"},
    "WS2M":{"title_tr":"Rüzgar hızı (2 m)","unit":"m/s"},
    "WS2M_MAX":{"title_tr":"Maks. rüzgar (2 m)","unit":"m/s"},
    "PRECTOTCORR":{"title_tr":"Toplam yağış (düz.)","unit":"mm/gün"},
    "TO3":{"title_tr":"Toplam ozon","unit":"DU"},
    "TS":{"title_tr":"Yüzey sıcaklığı","unit":"°C"},
    "DISTANCE_KM":{"title_tr":"Uzaklık (iklim pikseli)","unit":"km"},
    # Arazi & Gece ışığı
    "ELEVATION_M":{"title_tr":"Rakım","unit":"m"},
    "NIGHT_LIGHT":{"title_tr":"Gece ışığı","unit":"-"},
    # Toprak (HWSD)
    "FAO90_DESC":{"title_tr":"FAO-90 sınıfı","unit":"-"},
    "T_USDA_TEX_DESC":{"title_tr":"USDA doku (üst)","unit":"-"},
    "S_USDA_TEX_DESC":{"title_tr":"USDA doku (alt)","unit":"-"},
    "T_TEXTURE_DESC":{"title_tr":"Üst doku (coarse/medium/fine)","unit":"-"},
    "T_SAND":{"title_tr":"Kum (üst)","unit":"%"},
    "T_SILT":{"title_tr":"Silt (üst)","unit":"%"},
    "T_CLAY":{"title_tr":"Kil (üst)","unit":"%"},
    "S_SAND":{"title_tr":"Kum (alt)","unit":"%"},
    "S_SILT":{"title_tr":"Silt (alt)","unit":"%"},
    "S_CLAY":{"title_tr":"Kil (alt)","unit":"%"},
    "T_PH_H2O":{"title_tr":"pH (üst)","unit":"-"},
    "S_PH_H2O":{"title_tr":"pH (alt)","unit":"-"},
    "T_OC":{"title_tr":"Organik C (üst)","unit":"%"},
    "S_OC":{"title_tr":"Organik C (alt)","unit":"%"},
    "T_CEC_SOIL":{"title_tr":"CEC (üst)","unit":"cmol(+)/kg"},
    "S_CEC_SOIL":{"title_tr":"CEC (alt)","unit":"cmol(+)/kg"},
    "T_CEC_CLAY":{"title_tr":"CEC (kil, üst)","unit":"cmol(+)/kg"},
    "S_CEC_CLAY":{"title_tr":"CEC (kil, alt)","unit":"cmol(+)/kg"},
    "T_BS":{"title_tr":"Baz doygunluğu (üst)","unit":"%"},
    "S_BS":{"title_tr":"Baz doygunluğu (alt)","unit":"%"},
    "T_TEB":{"title_tr":"Toplam değişebilir baz (üst)","unit":"cmol(+)/kg"},
    "S_TEB":{"title_tr":"Toplam değişebilir baz (alt)","unit":"cmol(+)/kg"},
    "T_CACO3":{"title_tr":"Kireç CaCO₃ (üst)","unit":"%"},
    "S_CACO3":{"title_tr":"Kireç CaCO₃ (alt)","unit":"%"},
    "T_ECE":{"title_tr":"EC (üst)","unit":"dS/m"},
    "S_ECE":{"title_tr":"EC (alt)","unit":"dS/m"},
    "T_ESP":{"title_tr":"ESP (üst)","unit":"%"},
    "S_ESP":{"title_tr":"ESP (alt)","unit":"%"},
    "AWC_MM_PER_M":{"title_tr":"Kullanılabilir su (AWC)","unit":"mm/m"},
    "DRAINAGE_DESC":{"title_tr":"Drenaj","unit":"-"},
    "MU_GLOBAL":{"title_tr":"Harita birimi (MU)","unit":"-"},
}

def meta_of(key: str):
    k = key
    # *_grpN takılarını iç başlıktan ayır
    base = k.replace("_grp1","").replace("_grp2","").replace("_grp3","").replace("_grp4","")
    m = VAR_META.get(k) or VAR_META.get(base.upper()) or VAR_META.get(base)
    if m: return m["title_tr"], m.get("unit","-")
    return base, "-"

def format_value(v):
    if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))): return "-"
    try:
        f = float(v)
        if abs(f - round(f)) < 1e-9: return f"{int(round(f))}"
        return f"{f:.2f}"
    except Exception:
        return str(v)

# =========================
# 3) Veri kaynakları: İklim CSV + raster + HWSD
# =========================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def load_climate_nearest(csv_path: Path, lat: float, lon: float) -> dict:
    if not csv_path.exists():
        raise FileNotFoundError(f"İklim CSV bulunamadı: {csv_path}")
    df = pd.read_csv(csv_path)
    if not {"latitude","longitude"}.issubset(df.columns):
        raise ValueError("İklim CSV’de 'latitude' ve 'longitude' sütunları yok.")
    dist = haversine(lat, lon, df["latitude"].values, df["longitude"].values)
    i = int(np.argmin(dist))
    row = df.iloc[i].to_dict()
    row["DISTANCE_KM"] = float(dist[i])
    return row

def sample_raster(path: Path, lon: float, lat: float):
    try:
        import rasterio
        if not path.exists(): return None
        with rasterio.open(path) as ds:
            r, c = ds.index(lon, lat)
            val = ds.read(1)[r, c]
            if ds.nodata is not None and val == ds.nodata: return None
            return float(val)
    except Exception:
        return None

# ---- HWSD bağlayıcı (opsiyonel) ----
def load_soil_env(lat: float, lon: float) -> Optional[dict]:
    try:
        import rasterio, pyodbc
    except Exception:
        return None
    if not HWSD_MDB.exists() or not HWSD_RAS.exists():
        return None
    # MU_GLOBAL bul
    try:
        with rasterio.open(HWSD_RAS) as src:
            r, c = src.index(lon, lat)
            mu = int(src.read(1)[r, c])
        if mu <= 0:
            return None
    except Exception:
        return None
    # Access tablolar
    def _read(table):
        cn = pyodbc.connect(f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(HWSD_MDB).replace('\\','/')};")
        try:
            return pd.read_sql(f"SELECT * FROM {table}", cn)
        finally:
            cn.close()
    def _norm(df):
        df = df.copy(); df.columns = [c.strip().upper() for c in df.columns]; return df

    try:
        hwsd = _norm(_read("HWSD_DATA"))
    except Exception:
        return None

    # lookuplar
    def _lut(df, out_code, out_desc):
        if df is None: return None
        cols = set(df.columns)
        code = "CODE" if "CODE" in cols else None
        desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
        if code and desc:
            return df.rename(columns={code:out_code, desc:out_desc})[[out_code,out_desc]]
        return None

    def _safe(name):
        try: return _norm(_read(name))
        except Exception: return None

    tex   = _lut(_safe("D_TEXTURE"), "T_TEXTURE", "T_TEXTURE_DESC")
    utexT = _lut(_safe("D_USDA_TEX_CLASS"), "T_USDA_TEX_CLASS", "T_USDA_TEX_DESC")
    utexS = _lut(_safe("D_USDA_TEX_CLASS"), "S_USDA_TEX_CLASS", "S_USDA_TEX_DESC")
    awc   = _lut(_safe("D_AWC"), "AWC_CLASS", "AWC_MM_PER_M")
    drn   = _lut(_safe("D_DRAINAGE"), "DRAINAGE", "DRAINAGE_DESC")
    sym90 = _lut(_safe("D_SYMBOL90"), "SU_CODE90", "FAO90_DESC")

    df = hwsd
    for cond, lut, key in [
        ("T_TEXTURE", tex, "T_TEXTURE"),
        ("T_USDA_TEX_CLASS", utexT, "T_USDA_TEX_CLASS"),
        ("S_USDA_TEX_CLASS", utexS, "S_USDA_TEX_CLASS"),
        ("AWC_CLASS", awc, "AWC_CLASS"),
        ("DRAINAGE", drn, "DRAINAGE"),
        ("SU_CODE90", sym90, "SU_CODE90"),
    ]:
        if (cond in df.columns) and (lut is not None):
            df = df.merge(lut, on=key, how="left")

    num_cols = [
        "AWC_MM_PER_M","T_PH_H2O","S_PH_H2O","T_OC","S_OC",
        "T_CLAY","T_SILT","T_SAND","S_CLAY","S_SILT","S_SAND",
        "T_ECE","S_ECE","T_ESP","S_ESP","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB","T_CACO3","S_CACO3",
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")

    # dominant bileşen (SEQ, SHARE)
    if not {"MU_GLOBAL","SEQ","SHARE"}.issubset(df.columns):
        return None
    dom = (df.sort_values(["MU_GLOBAL","SEQ","SHARE"], ascending=[True,True,False])
             .groupby("MU_GLOBAL", as_index=False).first())
    row = dom.loc[dom["MU_GLOBAL"]==mu]
    if row.empty: return None
    return row.iloc[0].to_dict()

# =========================
# 4) ENV inşa + UI bas
# =========================
def build_env(lat: float, lon: float) -> dict:
    env = {}
    env["USER_LAT"] = float(lat); env["USER_LON"] = float(lon)

    clim = load_climate_nearest(CLIMATE_CSV, lat, lon)  # tüm sütunlar
    env.update(clim)
    if "latitude" in clim and "longitude" in clim:
        env["GRID_LAT"] = float(clim["latitude"])
        env["GRID_LON"] = float(clim["longitude"])

    elev  = sample_raster(ELEV_TIF,  lon, lat)
    night = sample_raster(LIGHT_TIF, lon, lat)
    if elev  is not None: env["ELEVATION_M"] = elev
    if night is not None: env["NIGHT_LIGHT"] = night

    soil = load_soil_env(lat, lon)
    if soil: env.update(soil)

    # skor modülü için alias
    def _pick(d,*keys):
        for k in keys:
            if k in d: return d[k]
        return None
    env['T2M_grp2']               = _pick(env,'T2M_grp2','T2M_GRP2','T2M')
    env['T2M_MIN_grp2']           = _pick(env,'T2M_MIN_grp2','T2M_MIN_GRP2','T2M_MIN')
    env['T2M_MAX_grp2']           = _pick(env,'T2M_MAX_grp2','T2M_MAX_GRP2','T2M_MAX')
    env['RH2M_grp2']              = _pick(env,'RH2M_grp2','RH2M_GRP2','RH2M')
    env['ALLSKY_SFC_SW_DWN_grp1'] = _pick(env,'ALLSKY_SFC_SW_DWN_grp1','ALLSKY_SFC_SW_DWN_GRP1','ALLSKY_SFC_SW_DWN')
    env['PRECTOTCORR_grp4']       = _pick(env,'PRECTOTCORR_grp4','PRECTOTCORR_GRP4','PRECTOTCORR')
    env['WS2M_MAX_grp3']          = _pick(env,'WS2M_MAX_grp3','WS2M_MAX_GRP3','WS2M_MAX')
    return env

def print_env_ui(env: dict):
    # Üst bilgi
    print(f"\n📍 Konum (kullanıcı): {format_value(env.get('USER_LAT'))}, {format_value(env.get('USER_LON'))}\n")
    # İklim hücresi & mesafe
    if env.get("GRID_LAT") is not None and env.get("GRID_LON") is not None:
        print("📌 Konum")
        print(f"- İklim hücresi enlem (GRID_LAT): {format_value(env['GRID_LAT'])} °")
        print(f"- İklim hücresi boylam (GRID_LON): {format_value(env['GRID_LON'])} °")
        if 'DISTANCE_KM' in env:
            print(f"- Uzaklık (iklim pikseli) (DISTANCE_KM): {format_value(env['DISTANCE_KM'])} km")
        print()

    # Tüm anahtarları kategorize edip yaz
    icons = {"Konum":"📌","İklim":"🌦️","Arazi":"⛰️","Gece Işığı":"🌃","Toprak":"🌱","Özet":"🧾"}
    def _key_sort(k):
        cat = category_of(k)
        return (CATEGORY_ORDER.index(cat) if cat in CATEGORY_ORDER else 99, k)
    keys = sorted(env.keys(), key=_key_sort)

    current = None
    for k in keys:
        # zaten üstte yazdıklarımızı tekrar etmeyelim
        if k in {"USER_LAT","USER_LON","GRID_LAT","GRID_LON"}:
            continue
        cat = category_of(k)
        # Konum kategorisinde latitude/longitude (grid) zaten gösterildi
        if cat == "Konum" and k.upper() in {"LATITUDE","LONGITUDE"}:
            continue
        # Özet kategorisinde sadece DISTANCE_KM kalsın
        if cat == "Özet" and k != "DISTANCE_KM":
            continue

        title, unit = meta_of(k)
        # CLRSKY_DAYS birimini veri büyüklüğüne göre gün/ay ↔ gün/yıl
        if k.startswith("CLRSKY_DAYS"):
            v = env.get(k)
            if isinstance(v,(int,float)) and float(v) > 31:
                unit = "gün/yıl"

        if cat != current:
            print(f"{icons.get(cat,'•')} {cat}")
            current = cat

        unit_sfx = f" {unit}" if unit and unit != "-" else ""
        print(f"- {title} ({k}): {format_value(env.get(k))}{unit_sfx}")
    print()

# =========================
# 5) Uygunluk skoru (modüller)
# =========================
def _clip01(x): x=float(x); return 0.0 if x<0 else (1.0 if x>1 else x)
def _presence(val): return (val is not None) and (str(val).strip() != "")
def _mean_safe(vals): vals=[float(v) for v in vals if _presence(v)]; return sum(vals)/len(vals) if vals else None
def _trapezoid_score(x,a,b,c,d):
    if any(v is None for v in [x,a,b,c,d]): return None
    x=float(x); a,b,c,d = float(a),float(b),float(c),float(d)
    if a>=b or b>c or c>=d: return None
    if x<=a or x>=d: return 0.0
    if b<=x<=c: return 100.0
    if a<x<b: return 100.0*(x-a)/(b-a)
    return 100.0*(d-x)/(d-c)

def suitability_score(crop, env, weights=None, params=None):
    W={'thermal':12,'frost':6,'heat':6,'rad':8,'rh':8,'water':15,'ph':8,'ec':8,'soilphys':5,'taw':4,'esp':3,'caco3':3,'cec':2,'elev':5,'wind':3,'night':2}
    if isinstance(weights, dict): W.update(weights)
    P={'rh_opt':60.0,'rh_span':30.0,'rad_min':1.2,'rad_max':2.8,'frost_band':5.0,'heat_band':5.0,'taw_ref_default':120.0}

    modules, usedW = {}, {}
    Tavg=env.get('T2M_grp2'); Tmin=env.get('T2M_MIN_grp2'); Tmax=env.get('T2M_MAX_grp2')
    tmin_abs=crop.get('tmin_abs'); topt_min=crop.get('topt_min'); topt_max=crop.get('topt_max'); tmax_abs=crop.get('tmax_abs')

    if all(_presence(v) for v in [Tavg,tmin_abs,topt_min,topt_max,tmax_abs]):
        modules['thermal']=_trapezoid_score(Tavg,tmin_abs,topt_min,topt_max,tmax_abs); usedW['thermal']=W['thermal']
    if all(_presence(v) for v in [Tmin,tmin_abs]):
        modules['frost']=100.0 if float(Tmin)>=float(tmin_abs) else max(0.0, 100.0-100.0*(abs(float(tmin_abs)-float(Tmin))/P['frost_band'])); usedW['frost']=W['frost']
    if all(_presence(v) for v in [Tmax,tmax_abs]):
        modules['heat']=100.0 if float(Tmax)<=float(tmax_abs) else max(0.0, 100.0-100.0*(abs(float(Tmax)-float(tmax_abs))/P['heat_band'])); usedW['heat']=W['heat']

    R=env.get('ALLSKY_SFC_SW_DWN_grp1')
    if _presence(R):
        modules['rad']=100.0*_clip01((float(R)-P['rad_min'])/max(1e-6,(P['rad_max']-P['rad_min']))); usedW['rad']=W['rad']
    RH=env.get('RH2M_grp2')
    if _presence(RH):
        modules['rh']=100.0*_clip01(1.0-((float(RH)-P['rh_opt'])/P['rh_span'])**2); usedW['rh']=W['rh']

    Pmm=env.get('PRECTOTCORR_grp4'); kc_avg=_mean_safe([crop.get('kc_initial'),crop.get('kc_mid'),crop.get('kc_end')])
    ETc=env.get('ETc'); ET0=env.get('ET0'); AWC=env.get('AWC_MM_PER_M'); Zr=crop.get('root_depth_m')
    if _presence(Pmm) and _presence(kc_avg) and (_presence(ETc) or _presence(ET0)) and _presence(AWC) and _presence(Zr):
        if not _presence(ETc): ETc=float(ET0)*float(kc_avg)
        deficit=max(0.0, float(ETc)-float(Pmm))
        TAW=float(AWC)*float(Zr); denom=max(1.0, TAW/15.0)
        modules['water']=100.0*_clip01(1.0-deficit/denom); usedW['water']=W['water']

    soil_pH=env.get('T_PH_H2O'); pH_min=crop.get('pH_min'); pH_max=crop.get('pH_max')
    if all(_presence(v) for v in [soil_pH,pH_min,pH_max]):
        a=float(pH_min)-0.5; b=float(pH_min); c=float(pH_max); d=float(pH_max)+0.5
        modules['ph']=_trapezoid_score(float(soil_pH),a,b,c,d); usedW['ph']=W['ph']

    soil_EC=env.get('T_ECE'); ec_thr = crop.get('ece_threshold_dSm') if 'ece_threshold_dSm' in crop else crop.get('ece_threshold_dsm')
    if all(_presence(v) for v in [soil_EC,ec_thr]):
        thr=max(0.1,float(ec_thr)); modules['ec']=100.0*_clip01(1.0-float(soil_EC)/thr); usedW['ec']=W['ec']

    tex_ok=(crop.get('texture_ok') or "").lower().replace(" ","")
    tex_ok_set=set([t.strip().lower() for t in tex_ok.split(",") if t.strip()])
    tex_env=(env.get('T_USDA_TEX_DESC') or "").strip().lower().replace(" ","")
    drain_pref=(crop.get('drainage_preference') or "").strip().lower()
    drain_env =(env.get('DRAINAGE_DESC') or "").strip().lower()

    score_tex=None
    if tex_env:
        if tex_env in tex_ok_set: score_tex=100.0
        else:
            neigh={'loam':{'sandy_loam','silt_loam','clay_loam'},'sandy_loam':{'loam'},'silt_loam':{'loam'},'clay_loam':{'loam'},
                   'sandy_clay_loam':{'clay_loam','sandy_loam'},'silty_clay_loam':{'clay_loam','silt_loam'}}
            score_tex=60.0 if any((k in tex_ok_set and tex_env in neigh.get(k,set())) for k in tex_ok_set) else 0.0

    def _norm_drain(s):
        s=s.lower()
        if 'well' in s and 'moderate' not in s: return 'well'
        if 'moderately' in s: return 'moderately well'
        if 'very poorly' in s: return 'very poorly'
        if 'poorly' in s: return 'poorly'
        if 'somewhat' in s: return 'somewhat poorly'
        return None

    score_drain=None
    if drain_pref and drain_env:
        dkey=_norm_drain(drain_env); dmap={'well':100,'moderately well':70,'somewhat poorly':40,'poorly':0,'very poorly':0}
        score_drain=dmap.get(dkey,70.0)

    if score_tex is not None or score_drain is not None:
        parts,wsum=[],0.0
        if score_tex   is not None: parts.append((score_tex,0.6)); wsum+=0.6
        if score_drain is not None: parts.append((score_drain,0.4)); wsum+=0.4
        modules['soilphys']=sum(s*w for s,w in parts)/(wsum if wsum else 1.0); usedW['soilphys']=W['soilphys']

    if _presence(AWC) and _presence(Zr):
        TAW=float(AWC)*float(Zr); ref=float((params or {}).get('taw_ref_default',120.0))
        modules['taw']=100.0*_clip01(TAW/ref); usedW['taw']=W['taw']

    ESP=env.get('T_ESP') or env.get('ESP')
    if _presence(ESP):
        modules['esp']=100.0*_clip01(1.0-float(ESP)/8.0); usedW['esp']=W['esp']

    CACO3=env.get('T_CACO3') or env.get('S_CACO3') or env.get('CACO3')
    if _presence(CACO3):
        modules['caco3']=100.0*_clip01(1.0-float(CACO3)/10.0); usedW['caco3']=W['caco3']

    CEC=env.get('T_CEC_SOIL')
    if _presence(CEC):
        CEC=float(CEC); modules['cec']=40.0 if CEC<8.0 else (70.0 if CEC<12.0 else 100.0); usedW['cec']=W['cec']

    elev=env.get('ELEVATION_M'); elev_min=crop.get('elevation_min'); elev_max=crop.get('elevation_max')
    if _presence(elev):
        e=float(elev)
        if _presence(elev_min) and _presence(elev_max):
            a=float(elev_min)-200.0; b=float(elev_min); c=float(elev_max); d=float(elev_max)+200.0
            modules['elev']=_trapezoid_score(e,a,b,c,d)
        else:
            modules['elev']=100.0 if e<1500 else (70.0 if e<2000 else (40.0 if e<2500 else 0.0))
        usedW['elev']=W['elev']

    WSMAX=env.get('WS2M_MAX_grp3')
    if _presence(WSMAX):
        modules['wind']=100.0*_clip01(1.0-float(WSMAX)/15.0); usedW['wind']=W['wind']

    NL=env.get('NIGHT_LIGHT')
    if _presence(NL):
        modules['night']=100.0*_clip01(float(NL)/5.0); usedW['night']=W['night']

    if not usedW: return {'score': None, 'modules': modules, 'used_weights': usedW}
    wsum=float(sum(usedW.values())); total=0.0
    for k, sc in modules.items():
        if sc is None: continue
        wk=usedW.get(k,0.0)/wsum; total += wk*float(sc)
    return {'score': round(total,2), 'modules': {k:round(v,2) for k,v in modules.items()}, 'used_weights': usedW}

# =========================
# 6) Bitkileri yükle + sıralama
# =========================
def load_crops(crops_csv_path: Path) -> pd.DataFrame:
    if not crops_csv_path.exists():
        raise FileNotFoundError(f"Bitki CSV bulunamadı: {crops_csv_path}")
    df = pd.read_csv(crops_csv_path)
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def row_to_crop_dict(row: pd.Series) -> dict:
    d = row.to_dict()
    if 'ece_threshold_dsm' in d and 'ece_threshold_dSm' not in d:
        d['ece_threshold_dSm'] = d['ece_threshold_dsm']
    if not str(d.get('texture_ok','')).strip():
        d['texture_ok'] = ''
    return d

def weakest_modules(mod_dict, n=2):
    if not mod_dict: return []
    items = [(k,v) for k,v in mod_dict.items() if v is not None]
    if not items: return []
    items.sort(key=lambda x: x[1])
    return [f"{k}:{v:.0f}" for k,v in items[:n]]

def score_and_rank(env: dict, crops_df: pd.DataFrame, top_k: int = 10):
    results = []
    for _, row in crops_df.iterrows():
        crop = row_to_crop_dict(row)
        res = suitability_score(crop, env)
        if res.get('score') is None: 
            continue
        results.append({'crop': crop.get('crop'),
                        'common_name_tr': crop.get('common_name_tr'),
                        'score': res['score'],
                        'modules': res.get('modules', {})})
    if not results:
        print("⚠️ Skor üretilemedi (gerekli çevre/bitki alanları eksik olabilir).")
        return

    results = sorted(results, key=lambda x: x['score'], reverse=True)
    top = results[:max(1, top_k)]

    print("— Ürün Uygunluk Skoru (0–100) —")
    print(f"{'Ürün':<28} {'Skor':>5}  {'Neyi sınırlıyor? (en zayıf 2 modül)':<40}")
    print("-"*80)
    for r in top:
        trname = (r['common_name_tr'] or r['crop'])
        wmods = ", ".join(weakest_modules(r['modules'], n=2)) or "-"
        print(f"{trname:<28} {r['score']:>5.1f}  {wmods:<40}")

    first3 = [ (r['common_name_tr'] or r['crop']) for r in top[:3] ]
    print("\nÖnerilen ilk 3: " + ", ".join(first3))
    print(f"🏆 En uygun ürün: {first3[0]}\n")

# =========================
# 7) CLI — konum iste, çalıştır
# =========================
def ask_for_location():
    try:
        line = input("Konum girin (Google Maps linki YA DA 'lat,lon'):  ").strip()
        if not line:
            raise ValueError("Boş giriş.")
        return line
    except (EOFError, KeyboardInterrupt):
        print("\nİptal edildi."); sys.exit(1)

def main():
    q = ask_for_location()
    try:
        lat, lon = parse_any_location(q)
    except Exception as e:
        print(f"⚠️ Konum çözümlenemedi: {e}"); sys.exit(2)

    print(f"\n📍 Konum: {lat:.5f}, {lon:.5f}\nℹ️ Veriler toplanıyor...\n")
    try:
        env = build_env(lat, lon)
    except Exception as e:
        print(f"❌ ENV hatası: {type(e).__name__}: {e}")
        print(f"👉 Kontrol: CLIMATE_CSV={CLIMATE_CSV}")
        sys.exit(1)

    # 1) TÜM VERİLER — UI gibi, yorum yok
    print_env_ui(env)

    # 2) SKORLAMA
    try:
        crops_df = load_crops(CROPS_CSV)
        print("🔎 Uygunluk hesaplanıyor ve sıralanıyor...\n")
        score_and_rank(env, crops_df, top_k=10)
        print("— bitti —")
    except Exception as e:
        print(f"⚠️ Skorlama hatası: {type(e).__name__}: {e}")
        print(f"👉 Kontrol: CROPS_CSV={CROPS_CSV}")

if __name__ == "__main__":
    main()

import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy", category=UserWarning)


Konum girin (Google Maps linki YA DA 'lat,lon'):   38.55420515917048, 38.707943735001976



📍 Konum: 38.55421, 38.70794
ℹ️ Veriler toplanıyor...


📍 Konum (kullanıcı): 38.55, 38.71

📌 Konum
- İklim hücresi enlem (GRID_LAT): 38.50 °
- İklim hücresi boylam (GRID_LON): 38.50 °
- Uzaklık (iklim pikseli) (DISTANCE_KM): 19.07 km

🌦️ İklim
- PAR (tümü) (ALLSKY_SFC_PAR_TOT_grp1): 0.86 MJ/m²/gün
- Kısa dalga (tümü) (ALLSKY_SFC_SW_DWN_grp1): 1.98 kWh/m²/gün
- Bulutluluk (gündüz) (CLOUD_AMT_DAY_grp1): 73.09 %
- Bulutluluk (gece) (CLOUD_AMT_NIGHT_grp1): 73.45 %
- Bulutluluk (CLOUD_AMT_grp1): 73.29 %
- Açık gün sayısı (CLRSKY_DAYS_grp1): 7 gün/ay
- Kısa dalga (açık gök) (CLRSKY_SFC_SW_DWN_grp1): 3.16 kWh/m²/gün
- Uzaklık (iklim pikseli) (DISTANCE_KM): 19.07 km
- Toplam yağış (düz.) (PRECTOTCORR_grp4): 1.58 mm/gün
- Yüzey basıncı (PS_grp3): 88.23 kPa
- Özgül nem (2 m) (QV2M_grp2): 3.12 g/kg
- Bağıl nem (2 m) (RH2M_grp2): 81.33 %
- Denize indirgenmiş basınç (SLP_grp3): 101.94 kPa
- Çiy noktası (T2MDEW_grp2): -4.81 °C
- Yaş termometre (T2MWET_grp2): -3.24 °C
- Maks. sıcaklık (T2M_MAX_grp2):